1. EDA
1. Data Preprocess
1. Single Model
1. Feature Engineering
1. Stacking
1. submit

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
train = pd.read_csv('/kaggle/input/sf-crime/train.csv.zip')
test = pd.read_csv('/kaggle/input/sf-crime/test.csv.zip', index_col='Id')


In [ ]:
submission = pd.read_csv('/kaggle/input/sf-crime/sampleSubmission.csv.zip',index_col='Id')
submission

In [ ]:
train

In [ ]:
test

In [ ]:
Tename = list(test)
Tename

In [ ]:
df_train = train[Tename]
df_train

In [ ]:
df = pd.concat((df_train,test))

In [ ]:
df

In [ ]:
target = train['Category']

In [ ]:
target.value_counts()

In [ ]:
from sklearn.preprocessing import LabelEncoder

LB = LabelEncoder() 
target = LB.fit_transform(target)
print(LB.classes_)

In [ ]:
target

In [ ]:
date = pd.to_datetime(df['Dates'])
df['Date'] = date.dt.date
df['Year'] = date.dt.year
df['Month'] = date.dt.month
df['Day'] = date.dt.day
df['Hour'] = date.dt.hour
df

In [ ]:
df.drop('Dates', axis=1, inplace=True)

In [ ]:
df

In [ ]:
year = df.groupby('Year').count().iloc[:,0]
month = df.groupby('Month').count().iloc[:,0]
hour = df.groupby('Hour').count().iloc[:,0]
dayofweek = df.groupby('DayOfWeek').count().iloc[:, 0]

figure, axs = plt.subplots(2,2, figsize = (15,10))

sns.barplot(x=year.index, y= year,ax = axs[0][0])
sns.barplot(x=month.index, y= month,ax = axs[0][1])
sns.barplot(x=hour.index, y= hour,ax = axs[1][0])
sns.barplot(x=dayofweek.index, y= dayofweek,ax = axs[1][1])

In [ ]:
lb = LabelEncoder()
df['PdDistrict'] = lb.fit_transform(df["PdDistrict"])
df['DayOfWeek'] = lb.fit_transform(df["DayOfWeek"])

In [ ]:
df["Address"].value_counts().head(20)

In [ ]:
df['Block'] = df['Address'].str.contains('Block')
df['ST'] = df['Address'].str.contains('ST')

In [ ]:
df

In [ ]:
df['Block'] = lb.fit_transform(df["Block"])
df['ST'] = lb.fit_transform(df["ST"])

In [ ]:
df

In [ ]:
df.drop("Address", axis =1, inplace = True)

샌프란시스코 위도(x): 37.7272, 경도(y): -123.032

In [ ]:
print(df["X"].min(), df["X"].max())
print(df["Y"].min(), df["Y"].max())

위도	32°30'N ~ 42°N   
경도	114°8'W ~ 124°24'W

In [ ]:
print(len(df.loc[df["X"] >= -120.5]))
print(len(df.loc[df["Y"] >= 90]))

In [ ]:
df.loc[df["X"] >= -120.5, "X"] = df[df["X"] < -120.5]["X"].median()
df.loc[df["Y"] >= 90, "Y"] = df[df["Y"] < -90]["Y"].median()

In [ ]:
df['X+Y'] = df['X'] + df['Y']
df['X-Y'] = df['X'] - df['Y']
df

In [ ]:
df.drop("Date", axis = 1, inplace = True)
df

III. 모델링



In [ ]:
df

In [ ]:
new_train = df[:train.shape[0]]
new_test = df[train.shape[0]:]

In [ ]:
new_train

In [ ]:
len(target)

In [ ]:
new_test

In [ ]:
del train, test

In [ ]:
new_train.shape

In [ ]:
target.shape

https://nurilee.com/2020/04/03/lightgbm-definition-parameter-tuning/

In [ ]:
!nvidia-smi

In [ ]:
import time
import xgboost as xgb
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split


clf = xgb.XGBClassifier(tree_method='gpu_hist', predictor='gpu_predictor', objective ='multi:softprob')

param_grid = {
        'silent': [False],
        'max_depth': [6, 10, 15, 20],
        'learning_rate': [0.001, 0.01, 0.1, 0.2, 0,3],
        'subsample': [0.5, 0.6, 0.7, 0.8, 0.9, 1.0],
        'colsample_bytree': [0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0],
        'colsample_bylevel': [0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0],
        'min_child_weight': [0.5, 1.0, 3.0, 5.0, 7.0, 10.0],
#         'max_bin' : [100, 200, 300 ,400, 500],
        'gamma': [0, 0.25, 0.5, 1.0],
        'reg_lambda': [0.1, 1.0, 5.0, 10.0, 50.0, 100.0],
        'n_estimators': [100]}

# fit_params = {'eval_metric': 'mlogloss',  'early_stopping_rounds': 10,   'eval_set': [(x_valid, y_valid)]}

rs_clf = RandomizedSearchCV(clf, param_grid, n_jobs=-1, verbose=1, cv=2,
                            scoring='neg_log_loss', refit=False, random_state=42) # n_iter=10,

print("Randomized search")
search_time_start = time.time()

rs_clf.fit(new_train, target)
print("Randomized search time:", time.time() - search_time_start)

best_score = rs_clf.best_score_
best_params = rs_clf.best_params_
print("Best score: {}".format(best_score))
print("Best params: ")

for param_name in sorted(best_params.keys()):
    print('%s: %r' % (param_name, best_params[param_name]))

In [ ]:
from sklearn.model_selection import GridSearchCV

# XGBoost 분류기 생성
xgb_clf = xgb.XGBClassifier(tree_method='gpu_hist', predictor='gpu_predictor', objective ='multi:softprob')

xgb_param_grid={ 
                 'silent':[True],
                 'max_depth':[5,6,8],
                 'min_child_weight':[1,3,5],
                 'gamma':[0,1,2,3],
                 'nthread':[4],
                 'colsample_bytree':[0.5,0.8],
                 'colsample_bylevel':[0.9],
                 'subsample': [0.6, 0.8, 1.0],
                 'n_estimators':[50],
                 'random_state':[42]}


# Create a GridSearchCV object

hr_grid = GridSearchCV(estimator=xgb_clf,
                       param_grid=xgb_param_grid,
                       scoring='neg_log_loss',
                       n_jobs=-1,
                       cv=5,
                       return_train_score=True)

hr_grid.fit(new_train, target)

best_score = hr_grid.best_score_
# 최고성능을 내는 행을 찾아냄
best_row = hr_grid.best_index_

# 최적 초모수: max_depth, subsample
best_max_depth     = hr_grid.best_params_["max_depth"]
best_max_subsample = hr_grid.best_params_["subsample"]

nl = '\n'
print(f'예측모형성능(AUC):  \t {best_score:.3f}{nl}\
        인덱스:           \t {best_row}{nl}\
        max_depth:      \t {max_depth}{nl}\
        subsample:      \t {subsample}')

    'max_depth': 4,     
    'eta': 0.3,     
    'silent': 1,    
    'objective': 'multi:softprob',    
    'num_class': 39,   

In [ ]:
predictions = bst.predict(new_test)

In [ ]:
submission = pd.DataFrame(predictions,columns=LB.inverse_transform(np.linspace(0, 38, 39, dtype='int16')),index=new_test.index)
#submission.to_csv('LGB.csv', index_label='Id')

In [ ]:
import lightgbm as lgb
from bayes_opt import BayesianOptimization
from sklearn.model_selection import cross_val_score

def lgb_evaluate(                
                numLeaves,
                maxDepth,
#                 scaleWeight,
#                 minChildWeight,
                Subsample,
                colSam,
                train_x, train_y
                ):
    clf = lgb.LGBMClassifier(
        objective = 'multiclass',
        metric= 'multi_logloss',
#         eval_metric= 'logloss',
#         reg_alpha= 0,
#         reg_lambda= 2,
#         bagging_fraction= 0.999,
#         min_split_gain= 0,
#         min_child_samples= 10,
#         subsample_freq= 3,
#         subsample_for_bin= 50000,
#         n_estimators= 100,
#         tree_method = 'gpu_hist',
        device_type = 'gpu',
        num_leaves= int(numLeaves),
        max_depth= int(maxDepth),
#         scale_pos_weight= scaleWeight,
#         min_child_weight= minChildWeight,
        subsample= Subsample,
        colsample_bytree= colSam)
#     clf.fit(train_x, train_y)
    scores = cross_val_score(clf, train_x, train_y, scoring='neg_log_loss')  # cv=5,
    print(scores.mean())

    return scores.mean()
   
def bayesOpt(train_x, train_y):
    def dtc_crossval(numLeaves,
                    maxDepth,
#                     scaleWeight,
#                     minChildWeight,
                    subsample,
                    colSam):
        return lgb_evaluate(
            numLeaves= int(numLeaves),
            maxDepth= int(maxDepth),
#             scaleWeight= scaleWeight,
#             minChildWeight= minChildWeight,
            Subsample= subsample,
            colSam= colSam,
            train_x=train_x,
            train_y=train_y,
        )    
    
    lgbBO = BayesianOptimization(dtc_crossval,
                                 {'numLeaves':  (5, 50),
                                    'maxDepth': (2, 50),
#                                     'scaleWeight': (1, 10000),
#                                     'minChildWeight': (0.01, 70),
                                    'subsample': (0.4, 1),                                                
                                    'colSam': (0.4, 1)
                                  
                                            })
    

    lgbBO.maximize(init_points=5, n_iter=5)
    
    print("Final result:", lgbBO.max)
#   print(lgbBO.res['max'])
    
bayesOpt(new_train, target)